## Asymmetric Semantic Search

For asymmetric semantic search, you usually have a short query (like a question or some keywords) and you want to find a longer paragraph answering the query.

### Achtung

The language is important, some models focus on english, others need to be told which language to use. The one I used here is multilingual. It's slower but seems good.

In [1]:
import fitz # requires pymupdf
from tqdm.auto import tqdm # for progress bars, requires tqdm
import re

pdf_path = "./example.pdf"

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ")
    cleaned_text = re.sub(r' ! ', '', cleaned_text)
    cleaned_text = re.sub(r'-\s+', '', cleaned_text)
    cleaned_text = re.sub(r'\s\s+', ' ', cleaned_text)
    cleaned_text = re.sub(r'(\d)!', r'\1€', cleaned_text)
    cleaned_text = re.sub(r'!', '', cleaned_text)

    # note: this might be different for each doc (best to experiment)
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({
            "page_number": page_number + 1,
            "page_char_count": len(text),
            "page_word_count": len(text.split(" ")),
            "page_sentence_count_raw": len(text.split(". ")),
            "page_token_count": len(text) / 4,  # 1 token = ~4 chars
            "text": text
        })

    return pages_and_texts


In [2]:
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:3]

0it [00:00, ?it/s]

[{'page_number': 1,
  'page_char_count': 472,
  'page_word_count': 67,
  'page_sentence_count_raw': 6,
  'page_token_count': 118.0,
  'text': 'INF 0122 DOCUMENTI INFORMATIVI Documenti informativi relativi al contatto per la ricezione e trasmissione di ordini, nonché esecuzione per conto del Cliente, collocamento e servizi accessori. 1. Informativa Pre-contrattuale – cliente al dettaglio – ed. ottobre 2021 2. Informativa Privacy, ai sensi dell’art.13, del Regolamento UE n.679/2016 (regolamento europeo in materia di protezione dei dati personali “GDPR”) 3. Allegato Economico (Allegato 1) – costi e commissioni '},
 {'page_number': 2,
  'page_char_count': 3910,
  'page_word_count': 550,
  'page_sentence_count_raw': 18,
  'page_token_count': 977.5,
  'text': "1/12 PRE 0124 Directa Società di intermediazione Mobiliare per Azioni Iscritta all’albo delle SIM al n° 59 Codice fiscale e partita iva e iscrizione al registro delle imprese n° 06837440012 Sede legale: Via Bruno Buozzi n° 5 – 10121 To

In [4]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,1,472,67,6,118.00,INF 0122 DOCUMENTI INFORMATIVI Documenti infor...
1,2,3910,550,18,977.50,1/12 PRE 0124 Directa Società di intermediazio...
2,3,7265,1051,25,1816.25,"2/12 www.consob.it e/o richieste a CONSOB, 001..."
3,4,6761,954,26,1690.25,3/12 prelevati su richiesta del Cliente dirett...
4,5,7676,1119,30,1919.00,"4/12 schio. A rendimenti potenziali maggiori, ..."


In [5]:
from spacy.lang.it import Italian

nlp = Italian()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/
nlp.add_pipe("sentencizer")

for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)

    # Make sure all sentences are strings (by default they are Spans)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]

    # Count the sentences
    item["page_sentence_count_spacy"] = len(item["sentences"])


  0%|          | 0/17 [00:00<?, ?it/s]

In [6]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,17.00,17.00,17.00,17.00,17.00,17.00
mean,9.00,5680.65,819.29,24.71,1420.16,22.47
std,5.05,1950.48,279.84,10.18,487.62,9.54
min,1.00,472.00,67.00,6.00,118.00,6.00
25%,5.00,4632.00,715.00,18.00,1158.00,16.00
50%,9.00,6549.00,941.00,25.00,1637.25,25.00
75%,13.00,7255.00,1039.00,36.00,1813.75,28.00
max,17.00,7806.00,1119.00,39.00,1951.50,37.00


In [7]:
num_sentence_chunk_size = 7

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    # No overlap here, but could be useful to add at least one sentence of overlap
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/17 [00:00<?, ?it/s]

In [8]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,17.00,17.00,17.00,17.00,17.00,17.00,17.00
mean,9.00,5680.65,819.29,24.71,1420.16,22.47,3.65
std,5.05,1950.48,279.84,10.18,487.62,9.54,1.54
min,1.00,472.00,67.00,6.00,118.00,6.00,1.00
25%,5.00,4632.00,715.00,18.00,1158.00,16.00,3.00
50%,9.00,6549.00,941.00,25.00,1637.25,25.00,4.00
75%,13.00,7255.00,1039.00,36.00,1813.75,28.00,4.00
max,17.00,7806.00,1119.00,39.00,1951.50,37.00,6.00


In [9]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]

        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters

        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

  0%|          | 0/17 [00:00<?, ?it/s]

62

In [10]:
import random
random.sample(pages_and_chunks, k=3)

[{'page_number': 5,
  'sentence_chunk': "4/12 schio. A rendimenti potenziali maggiori, in linea generale, corrispondono anche rischi maggiori. Segue qui una breve enumerazione dei principali strumenti finanziari negoziati sui mercati, indipendentemente dal fatto che siano o meno messi a disposizione da Directa al momento attuale.1.1) Titoli di capitale e titoli di debito Occorre distinguere innanzitutto tra titoli di capitale (i titoli più diffusi di tale categoria sono le azioni) e titoli di debito (tra i più diffusi titoli di debito si ricordano le obbligazioni e i certificati di deposito), tenendo conto che: a) acquistando titoli di capitale si diviene soci della società emittente, partecipando per intero al rischio economico della medesima; chi investe in titoli azionari ha diritto a percepire annualmente sugli utili conseguiti nel periodo di riferimento il dividendo che l'assemblea dei soci deciderà di distribuire. L'assemblea dei soci può comunque stabilire di non distribuire alc

In [11]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,62.00,62.00,62.00,62.00
mean,8.60,1555.69,223.47,388.92
std,4.21,842.87,127.59,210.72
min,1.00,73.00,11.00,18.25
25%,5.25,1112.50,156.00,278.12
50%,8.00,1501.00,213.00,375.25
75%,11.75,1980.25,295.00,495.06
max,17.00,4632.00,740.00,1158.00


In [12]:
# Show random chunks with under 50 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 18.25 | Text: Mentre i certificates con leva si adattano maggiormente a investitori con


In [13]:
max_token_length = 500 # it's 512 but to be safe...
# in fact, we have to append query: and passage: to the beginning of the text
for row in df[df["chunk_token_count"] >= max_token_length].iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 679.0 | Text: 1/12 PRE 0124 Directa Società di intermediazione Mobiliare per Azioni Iscritta all’albo delle SIM al n° 59 Codice fiscale e partita iva e iscrizione al registro delle imprese n° 06837440012 Sede legale: Via Bruno Buozzi n° 5 – 10121 Torino Telefono: +39 011.530101 – fax: +39 011.530532 E.mail: directa@directa.it PEC: directasim@legalmail.it Capitale sociale Euro 7.500.000 interamente versato Aderente al Fondo Nazionale di Garanzia Contratto di: CONTRATTO PER LA RICEZIONE E TRASMISSIONE DI ORDINI, NONCHÉ ESECUZIONE PER CONTO DEL CLIENTE, COLLOCAMENTO E SERVIZI ACCESSORI INFORMATIVA PRE-CONTRATTUALE Cliente al dettaglio Edizione Ottobre 2021 Sezione A Informazioni su Directa e i suoi servizi Pag 1 Sezione B Informazioni concernenti la salvaguardia degli investimenti finanziari e delle somme di denaro della clientela Pag 3 Sezione C Informazioni sugli strumenti finanziari Pag 3 Sezione D Informazioni sugli oneri e sui costi Pag 10 Sezione E Informazioni pe

In [14]:
# slice the chunks that are too long
def split_chunks(pages_and_chunks, max_token_length: int) -> list[str]:
    pages_and_chunks_sliced_internal = []
    for item in tqdm(pages_and_chunks):
        if item["chunk_token_count"] >= max_token_length - 12:
            sentences = list(nlp(item["sentence_chunk"]).sents)

            first_half = []
            second_half = []
            if(len(sentences) == 1):
                # spit in half with 20% overlap
                midpoint = len(sentences[0]) // 2
                overlap = len(sentences[0]) * 20//100

                first_half = [sentences[0][:midpoint + overlap]]
                second_half = [sentences[0][midpoint - overlap:]]
            else:
                midpoint = len(sentences) // 2
                first_half = sentences[:midpoint]
                second_half = sentences[midpoint:]

            first_half_joined = "".join([str(sentence) for sentence in first_half]).strip()
            second_half_joined = "".join([str(sentence) for sentence in second_half]).strip()

            pages_and_chunks_sliced_internal.append({
                "page_number": item["page_number"],
                "sentence_chunk": first_half_joined,
                "chunk_char_count": len(first_half_joined),
                "chunk_word_count": len(first_half_joined.split(" ")),
                "chunk_token_count": len(first_half_joined) / 4
            })

            pages_and_chunks_sliced_internal.append({
                "page_number": item["page_number"],
                "sentence_chunk": second_half_joined,
                "chunk_char_count": len(second_half_joined),
                "chunk_word_count": len(second_half_joined.split(" ")),
                "chunk_token_count": len(second_half_joined) / 4
            })

        else: pages_and_chunks_sliced_internal.append(item)
    return pages_and_chunks_sliced_internal


pages_and_chunks_sliced = pages_and_chunks.copy()
while any(item["chunk_token_count"] >= max_token_length for item in pages_and_chunks_sliced):
    pages_and_chunks_sliced = split_chunks(pages_and_chunks_sliced, max_token_length)


  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/78 [00:00<?, ?it/s]

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/86 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

In [15]:
df = pd.DataFrame(pages_and_chunks_sliced)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,92.00,92.00,92.00,92.00
mean,9.49,1134.12,163.11,283.53
std,4.67,503.06,73.62,125.76
min,1.00,73.00,11.00,18.25
25%,5.75,787.25,111.50,196.81
50%,9.00,1237.00,176.00,309.25
75%,13.25,1497.50,217.00,374.38
max,17.00,1948.00,295.00,487.00


In [16]:
for row in df[df["chunk_token_count"] >= max_token_length].iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

#df.to_csv("./debug.csv", index=False)

In [19]:
from typing import List
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings

embedding_model_temp = SentenceTransformer('intfloat/multilingual-e5-large')
# print(embedding_model.get_max_seq_length)

# Extract the transformer and pooling layers from the original model
transformer = embedding_model_temp[0]  # Transformer layer
pooling = embedding_model_temp[1]

# skipping normalization layer (so that we can use cosine similarity for better search results)
embedding_model = SentenceTransformer(modules=[transformer, pooling])

class CustomEmbeddings(Embeddings):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        prefix = "passage: "
        return [embedding_model.encode(prefix + text, batch_size=32, normalize_embeddings=False) for text in texts]

    def embed_query(self, text: str) -> List[float]:
        prefix = "query: "
        return embedding_model.encode(prefix + text, normalize_embeddings=False)


In [20]:
from langchain.vectorstores import Chroma
from langchain.schema import Document

# Create a list of Documents with metadata
documents = [
    Document(
        page_content=row["sentence_chunk"],
        metadata={
            "page_number": row["page_number"],
            "chunk_char_count": row["chunk_char_count"],
            "chunk_word_count": row["chunk_word_count"],
            "chunk_token_count": row["chunk_token_count"]
        }
    )
    for _, row in df.iterrows()
]

vectorstore = Chroma.from_documents(documents, CustomEmbeddings())

In [22]:
import numpy as np
import pandas as pd

# Function to inspect and verify normalization of embeddings
def check_embedding_normalization(vectorstore, num_samples=100):
    # Get a sample of documents and embeddings
    sample_embeddings = []
    embeddings = vectorstore.get(include=['embeddings'])['embeddings']
    for i in range(num_samples):
        # Retrieve embedding of document i (assuming sequential IDs for simplicity)
        embedding = embeddings[i]  # Using private attribute here
        sample_embeddings.append(embedding)
    # Calculate and print the norms
    norms = [np.linalg.norm(embedding) for embedding in sample_embeddings]
    for i, norm in enumerate(norms):
        print(f"Embedding {i} norm: {norm}")

# Run the check
check_embedding_normalization(vectorstore)

df = pd.DataFrame(vectorstore.get(include=['embeddings'])['embeddings'])
df.head()

Embedding 0 norm: 28.920189284564916
Embedding 1 norm: 27.42166665541502
Embedding 2 norm: 28.53726760419476
Embedding 3 norm: 28.39743345186702
Embedding 4 norm: 27.962912926919177
Embedding 5 norm: 27.451907111667406
Embedding 6 norm: 27.53860509885427
Embedding 7 norm: 27.710475126903372
Embedding 8 norm: 27.414579958204
Embedding 9 norm: 27.254372884475927
Embedding 10 norm: 27.522026462263014
Embedding 11 norm: 27.146362711419442
Embedding 12 norm: 27.172680950290594
Embedding 13 norm: 27.896861627694303
Embedding 14 norm: 27.752392188634058
Embedding 15 norm: 27.62138969481705
Embedding 16 norm: 27.30625493225182
Embedding 17 norm: 27.40942605794435
Embedding 18 norm: 26.963059976560942
Embedding 19 norm: 27.413622151520983
Embedding 20 norm: 27.644565836306143
Embedding 21 norm: 27.554972192542014
Embedding 22 norm: 28.201857964442166
Embedding 23 norm: 27.138651764781294
Embedding 24 norm: 27.697008254103654
Embedding 25 norm: 27.263550925905868
Embedding 26 norm: 27.5973527314

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,0.571871,0.829987,-0.492226,-1.323852,0.734331,-1.088682,-0.595656,0.456774,1.402663,0.263563,...,-1.037626,-0.457798,0.692848,-0.063827,-0.563869,0.047277,0.621831,-0.527031,-1.734563,-0.285335
1,-0.049918,0.079582,0.067078,-0.989561,0.759164,-0.839253,-0.701803,1.297602,1.295123,0.664813,...,-1.047819,-0.475078,0.695177,-0.150724,-0.030163,0.278149,0.873583,-0.904569,-1.479388,-0.001628
2,-0.220622,0.151943,0.193604,-1.227387,0.911621,-0.338830,-1.081580,1.123100,1.410178,0.096975,...,-0.692260,-0.454297,1.164334,-0.692141,0.612112,0.628225,-0.309554,-0.478375,-1.386204,-0.219371
3,0.151602,0.375022,-0.307130,-1.351519,0.996929,-1.198945,0.110964,1.319898,0.621398,-0.346659,...,-1.153875,0.170254,0.349842,-0.404689,0.429035,0.122124,0.103099,-0.621213,-1.790604,0.075897
4,0.149271,-0.092246,-0.194368,-1.019592,0.893833,-0.203145,-0.698008,1.245406,1.496197,-0.482008,...,-0.520063,0.060892,-0.154036,0.151538,0.228079,0.768690,-0.024970,-1.047924,-1.583457,-0.338389


In [23]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

model = ChatOllama(
  model="llama3.2"
)

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever = vectorstore.as_retriever(search_kwargs={ "k": 2})


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "Quanto mi costa fare trading su EXM?"})

results

/Users/andreasimonecosta/mambaforge/lib/python3.12/site-packages/langchain/chains/combine_documents/reduce.py:10: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain.chains.combine_documents.base import BaseCombineDocumentsChain


{'input': 'Quanto mi costa fare trading su EXM?',
 'context': [Document(metadata={'chunk_char_count': 1345, 'chunk_token_count': 336.25, 'chunk_word_count': 217, 'page_number': 16}, page_content='COMMISSIONI DI TRADING SUI DIVERSI MERCATI EXM (ex MTA), EGM, MIV, ETFplus, GEM Profili alternativi: • Semplice: 5€ per ordine eseguito • Dinamica*: da 8 a 1,5€ • Variabile: 1,9 per mille per ordine eseguito, con un massimo di 18€ e un minimo di 1,5€ (il minimo è di 5€ per il mercato GEM) per ordini fino a 500.000€ ATFund • Unico profilo disponibile: 1,9 per mille per ordine eseguito, con un massimo di 200€ e un minimo di 5€ per ordini fino a 500.000€ SEDEX e EuroTLX certificati depositario Montetitoli Profili alternativi: • Semplice: 6€ per ordine eseguito • Dinamica*: da 9 a 2,5€ • Variabile: 1,9 per mille per ordine eseguito, con massimo di 18€ e minimo di 2€ per ordini fino a 500.000€ depositario Clearstream •Unico profilo disponibile: 7€ per ordine eseguito MOT Profili alternativi: •Sempl

In [24]:
print(results["context"][0].metadata)

{'chunk_char_count': 1345, 'chunk_token_count': 336.25, 'chunk_word_count': 217, 'page_number': 16}


In [25]:
rag_chain.invoke({"input": "Quanto mi costa fare una operazione fuori mercato?"})

{'input': 'Quanto mi costa fare una operazione fuori mercato?',
 'context': [Document(metadata={'chunk_char_count': 1345, 'chunk_token_count': 336.25, 'chunk_word_count': 217, 'page_number': 16}, page_content='COMMISSIONI DI TRADING SUI DIVERSI MERCATI EXM (ex MTA), EGM, MIV, ETFplus, GEM Profili alternativi: • Semplice: 5€ per ordine eseguito • Dinamica*: da 8 a 1,5€ • Variabile: 1,9 per mille per ordine eseguito, con un massimo di 18€ e un minimo di 1,5€ (il minimo è di 5€ per il mercato GEM) per ordini fino a 500.000€ ATFund • Unico profilo disponibile: 1,9 per mille per ordine eseguito, con un massimo di 200€ e un minimo di 5€ per ordini fino a 500.000€ SEDEX e EuroTLX certificati depositario Montetitoli Profili alternativi: • Semplice: 6€ per ordine eseguito • Dinamica*: da 9 a 2,5€ • Variabile: 1,9 per mille per ordine eseguito, con massimo di 18€ e minimo di 2€ per ordini fino a 500.000€ depositario Clearstream •Unico profilo disponibile: 7€ per ordine eseguito MOT Profili alter

In [26]:
rag_chain.invoke({"input": "Quali sono i profili alternativi disponibili per il trading?"})

{'input': 'Quali sono i profili alternativi disponibili per il trading?',
 'context': [Document(metadata={'chunk_char_count': 1349, 'chunk_token_count': 337.25, 'chunk_word_count': 207, 'page_number': 16}, page_content='IDEM EUREX mini FTSE MIB, Euro Bund, Euro Stoxx50, EuroSchatz, EuroBobl, EuroBuxl, EuroOATF e MiniDAX Profili alternativi: •Semplice: 4€ a contratto •Dinamica*: da 8 a 1,5€ a contratto FTSE MIB e FDAX, Long Btp, ShortBtp, FTSE MIB Div Profili alternativi: •Semplice: 6€ a contratto •Dinamica*: da 9 a 2,5€ a contratto micro FTSE MIB •Unico profilo disponibile: 2€ a contratto OPZIONI •Unico profilo disponibile: 2,5€ a contratto LMAX Exchange FOREX, Cross valutari, CFD su Commodities • Unico profilo disponibile: 0,003% del controvalore con minimo 1,5€ o 2€ CFD su Indici •Unico profilo disponibile: 1€ a contratto CFD su CRIPTOVALUTE • Unico profilo disponibile: 0,2% del controvalore con min.2€ Spectrum markets • Unico profilo disponibile: 5€ per ordine eseguito CME E-mini NQ